In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

from pprint import pprint
from zen_garden.postprocess.results import Results

from pathlib import Path
from os import chdir

In [3]:
# set path
path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/notebooks')
chdir(path)

In [4]:
# cost optimal solution
res_basic = Results("../data/outputs/03_example_CCTS")

/Users/davidbertschinger/Applications/PyCharm Professional Edition.app/Contents/plugins/python/helpers/pydev/pydevd_plugins/__init__.py:2: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydevd_plugins')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__('pkg_resources').declare_namespace(__name__)

/Users/davidbertschinger/Applications/PyCharm Professional Edition.app/Contents/plugins/python/helpers/pydev/pydevd_plugins/extensions/__init__.py:2: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydevd_plugins.extensions')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__('pkg_resources').decla

In [5]:
# show the deafult analysis settings
pprint(res_basic.results[None]["analysis"])

{'dataset': '/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/03_example_CCTS',
 'earliest_year_of_data': 1900,
 'folder_output': '/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/outputs',
 'header_data_inputs': {'set_capacity_types': 'capacity_type',
                        'set_carriers': 'carrier',
                        'set_conversion_technologies': 'technology',
                        'set_dependent_carriers': 'carrier',
                        'set_edges': 'edge',
                        'set_elements': 'element',
                        'set_failure_states': 'failure_states',
                        'set_input_carriers': 'carrier',
                        'set_lca_impact_categories': 'impact_category',
                        'set_location': 'location',
                        'set_nodes': 'node',
                        'set_output_carriers': 'carrier',
                        'set_retrofitting_technologies': 'technology',
                        'set_stor

In [6]:
# get flow transport and save as csv
res_basic.get_df('flow_transport', to_csv="flow_transport")

technology   edge                         time_operation
truck_batch  waste_1_CH-swiss_storage_CH  0                  0.000000
                                          1                 10.227416
                                          2                 10.227416
                                          3                 10.227416
                                          4                 10.227416
             waste_2_CH-swiss_storage_CH  0                  0.000000
                                          1                  7.635671
                                          2                  7.635671
                                          3                 19.807119
                                          4                 19.807119
             waste_1_CH-waste_2_CH        0                  0.000000
                                          1                  0.000000
                                          2                  0.000000
                                 

In [7]:
# get flow transport csv
df = pd.read_csv('flow_transport.csv')

In [8]:
df

,technology,edge,time_operation,flow_transport
0,truck_batch,waste_1_CH-swiss_storage_CH,0,0.000000
1,truck_batch,waste_1_CH-swiss_storage_CH,1,10.227416
2,truck_batch,waste_1_CH-swiss_storage_CH,2,10.227416
3,truck_batch,waste_1_CH-swiss_storage_CH,3,10.227416
4,truck_batch,waste_1_CH-swiss_storage_CH,4,10.227416
5,truck_batch,waste_2_CH-swiss_storage_CH,0,0.000000
6,truck_batch,waste_2_CH-swiss_storage_CH,1,7.635671
7,truck_batch,waste_2_CH-swiss_storage_CH,2,7.635671
8,truck_batch,waste_2_CH-swiss_storage_CH,3,19.807119
9,truck_batch,waste_2_CH-swiss_storage_CH,4,19.807119


In [9]:
# find unique technologies
technology = df['technology'].unique()
technology

array(['truck_batch'], dtype=object)

In [10]:
# create nominal flow transport files

for i in range(len(technology)):
     
    #Get table of one technology 
    tec = technology[i]
    x = df[df['technology'] == tec]
    
    #Set path
    parent_path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/03_example_CCTS_N-1')
    path = [file for file in Path(parent_path).rglob(tec)]
    path = path[0]
    filepath = Path(str(path)+'/nominal_flow_transport.csv')
    
    #Clean up table
    x = x.drop(columns='technology')
    x = x[x['flow_transport'] != 0]
    x = x.rename(columns={'time_operation': 'year'})
    x['year'] = x['year'] + 2025 
    
    #clean up table, differentiate between node and edge
    if path.parent.name == 'set_transport_technologies':
        x = x.rename(columns={'location': 'edge'})

    else:
        x = x.rename(columns={'location': 'node'})

    #Safe
    x.to_csv(filepath, index=False)